# Imports

In [1]:
%load_ext autoreload

In [2]:
# Load the Pandas libraries with alias 'pd'
import datetime

import numpy as np
import pandas as pd
from bokeh.io import output_notebook
from bokeh.layouts import grid
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource, LinearAxis, Range1d
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10_6, Category20_20
from bokeh.plotting import figure, show

from covid19.covid import (
    get_all_states_data,
    get_brazil_data,
    get_date_date_cases_greater_than,
    get_dia_de_contaminacao_array
)
from covid19.scrap import get_covid_data, read_datafile_from_disc

output_notebook(hide_banner=True)

import logging

logging.getLogger().setLevel(logging.INFO)

# Lendo os dados

In [3]:
# data = pd.read_excel("DT_PAINEL_COVIDBR_20200510.xlsx")
# data = read_datafile_from_disc("DT1_PAINEL_COVIDBR_20200512.xlsx")
data = get_covid_data()

# Entendendo os dados e separando dados do Brasil todo e de cada estado

As primeiras linhas possuem dados para o Brasil todo e a região é "Brasil".

In [4]:
print("Shape do dataset todo", data.shape)
data.head()

Shape do dataset todo (69650, 15)


,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,diaDaSemana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,0,NaN,NaN,Quarta
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,NaN,NaN,Quinta
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,NaN,NaN,Sexta
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,0,NaN,NaN,Sábado
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-01,10,210147125.0,2,0,NaN,NaN,Domingo


Note que a função que lê os dados adicionou uma coluna contendo o dia da semana.

Vamos separar o dataset em dois: um para o Brasil todo e outro para os estados. Note que colunas irrelevantes (como "município", por exemplo) foram removidas e 4 colunas novas foram adicionadas: **casosNovos**, **obitosNovos**, **diasDeContaminacao_1** e **diasDeContaminacao_100**.

In [5]:
data_brasil = get_brazil_data(data)
print("Shape do dataset do Brasil", data_brasil.shape)
data_brasil

Shape do dataset do Brasil (77, 12)


,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,diaDaSemana,casosNovos,obitosNovos,diasDeContaminacao_1,diasDeContaminacao_100
0,2020-02-26,9,210147125.0,1,0,NaN,NaN,Quarta,1,0,1,0
1,2020-02-27,9,210147125.0,1,0,NaN,NaN,Quinta,0,0,2,0
2,2020-02-28,9,210147125.0,1,0,NaN,NaN,Sexta,0,0,3,0
3,2020-02-29,9,210147125.0,2,0,NaN,NaN,Sábado,1,0,4,0
4,2020-03-01,10,210147125.0,2,0,NaN,NaN,Domingo,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,2020-05-08,19,210147125.0,145328,9897,59297.0,76134.0,Sexta,10222,751,73,56
73,2020-05-09,19,210147125.0,155939,10627,61685.0,83627.0,Sábado,10611,730,74,57
74,2020-05-10,20,210147125.0,162699,11123,64957.0,86619.0,Domingo,6760,496,75,58
75,2020-05-11,20,210147125.0,168331,11519,67384.0,89429.0,Segunda,5632,396,76,59


In [6]:
data_estados = get_all_states_data(data)
print("Shape do dataset dos estados", data_estados.shape)
print(data_estados.shape[0] / 77)

Shape do dataset dos estados (1626, 15)
21.116883116883116


In [7]:
# data_estados.groupby(by="estado").count()
data_estados

,regiao,estado,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,diaDaSemana,casosNovos,diasDeContaminacao_1,diasDeContaminacao_100,obitosNovos
887,Centro-Oeste,MS,50,2020-03-16,12,2778986.0,2,0,NaN,NaN,Segunda,2,1,0,0.0
888,Centro-Oeste,MS,50,2020-03-17,12,2778986.0,4,0,NaN,NaN,Terça,2,2,0,0.0
889,Centro-Oeste,MS,50,2020-03-18,12,2778986.0,7,0,NaN,NaN,Quarta,3,3,0,0.0
890,Centro-Oeste,MS,50,2020-03-19,12,2778986.0,7,0,NaN,NaN,Quinta,0,4,0,0.0
891,Centro-Oeste,MS,50,2020-03-20,12,2778986.0,9,0,NaN,NaN,Sexta,2,5,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69645,Sul,RS,43,2020-05-08,19,11377239.0,2447,91,NaN,NaN,Sexta,318,60,45,1.0
69646,Sul,RS,43,2020-05-09,19,11377239.0,2493,95,NaN,NaN,Sábado,46,61,46,4.0
69647,Sul,RS,43,2020-05-10,20,11377239.0,2542,97,NaN,NaN,Domingo,49,62,47,2.0
69648,Sul,RS,43,2020-05-11,20,11377239.0,2576,105,NaN,NaN,Segunda,34,63,48,8.0


# Primeiros casos

In [8]:
# data_brasil = data.groupby("data").sum()

print(f"Data do primeiro caso: {get_date_date_cases_greater_than(data, 0)}")
print(
    f"Data com mais de 10 casos: {get_date_date_cases_greater_than(data, 10)}")
print(
    f"Data com mais de 100 casos: {get_date_date_cases_greater_than(data, 100)}"
)
print(
    f"Data com mais de 1000 casos: {get_date_date_cases_greater_than(data, 1000)}"
)

Data do primeiro caso: 2020-02-26
Data com mais de 10 casos: 2020-03-06
Data com mais de 100 casos: 2020-03-14
Data com mais de 1000 casos: 2020-03-21


In [9]:
data_estados[data_estados.estado=="CE"].head()

,regiao,estado,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,diaDaSemana,casosNovos,diasDeContaminacao_1,diasDeContaminacao_100,obitosNovos
12679,Nordeste,CE,23,2020-03-17,12,9132078.0,5,0,NaN,NaN,Terça,5,1,0,0.0
12680,Nordeste,CE,23,2020-03-18,12,9132078.0,9,0,NaN,NaN,Quarta,4,2,0,0.0
12681,Nordeste,CE,23,2020-03-19,12,9132078.0,20,0,NaN,NaN,Quinta,11,3,0,0.0
12682,Nordeste,CE,23,2020-03-20,12,9132078.0,55,0,NaN,NaN,Sexta,35,4,0,0.0
12683,Nordeste,CE,23,2020-03-21,12,9132078.0,68,0,NaN,NaN,Sábado,13,5,0,0.0


# Vamos adicionar as colunas "casosNovos", "obitosNovos" e "diaDeContaminacao"

Primeiro adicionamos no dataframe do Brasil todo.

In [10]:
# Calcula casos novos a partir dos casos acumulados. Note que o primeiro valor na coluna "casosNovos" corresponde ao total de casos acumulados na primeira entrada
# data_brasil["casosNovos"] = np.diff(data_brasil.casosAcumulado, prepend=0)
# data_brasil["obitosNovos"] = np.diff(data_brasil.obitosAcumulado, prepend=0)

# data_brasil["diasDeContaminacao_1"] = get_dia_de_contaminacao_array(data_brasil, min_casos=1)
# data_brasil["diasDeContaminacao_100"] = get_dia_de_contaminacao_array(data_brasil, min_casos=100)

print(np.all(np.cumsum(data_brasil.casosNovos) == data_brasil.casosAcumulado))

True


In [11]:
# estados = data_estados.estado.unique()

# # Calcula novos casos a partir dos casos acumulados. Note que o primeiro valor na coluna "casosNovos" corresponde ao total de casos acumulados na primeira entrada
# data_estados["casosNovos"] = np.zeros(data_estados.shape[0], dtype=int)
# data_estados["diasDeContaminacao"] = np.zeros(data_estados.shape[0], dtype=int)
# for sigla in estados:
#     data_esse_estado = data_estados[data_estados.estado == sigla]
#     data_estados.loc[data_estados.estado == sigla,
#                      "casosNovos"] = np.diff(data_esse_estado.casosAcumulado,
#                                              prepend=0)
#     data_estados.loc[data_estados.estado == sigla,
#                      "obitosNovos"] = np.diff(data_esse_estado.obitosAcumulado,
#                                               prepend=0)
#     data_estados.loc[data_estados.estado == sigla,
#                      "diasDeContaminacao"] = get_dia_de_contaminacao_array(
#                          data_estados[data_estados.estado == sigla])

data_estados

,regiao,estado,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,diaDaSemana,casosNovos,diasDeContaminacao_1,diasDeContaminacao_100,obitosNovos
887,Centro-Oeste,MS,50,2020-03-16,12,2778986.0,2,0,NaN,NaN,Segunda,2,1,0,0.0
888,Centro-Oeste,MS,50,2020-03-17,12,2778986.0,4,0,NaN,NaN,Terça,2,2,0,0.0
889,Centro-Oeste,MS,50,2020-03-18,12,2778986.0,7,0,NaN,NaN,Quarta,3,3,0,0.0
890,Centro-Oeste,MS,50,2020-03-19,12,2778986.0,7,0,NaN,NaN,Quinta,0,4,0,0.0
891,Centro-Oeste,MS,50,2020-03-20,12,2778986.0,9,0,NaN,NaN,Sexta,2,5,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69645,Sul,RS,43,2020-05-08,19,11377239.0,2447,91,NaN,NaN,Sexta,318,60,45,1.0
69646,Sul,RS,43,2020-05-09,19,11377239.0,2493,95,NaN,NaN,Sábado,46,61,46,4.0
69647,Sul,RS,43,2020-05-10,20,11377239.0,2542,97,NaN,NaN,Domingo,49,62,47,2.0
69648,Sul,RS,43,2020-05-11,20,11377239.0,2576,105,NaN,NaN,Segunda,34,63,48,8.0


# Plot dados para o Brasil

In [12]:
# variable_to_plot_vec = np.array(['casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos'])
variable_to_plot_vec = np.array(['casosAcumulado', 'obitosAcumulado'])

# We only want values from this date in the plot
date_threshold = get_date_date_cases_greater_than(data, 100)

In [13]:
source_brasil = ColumnDataSource(data_brasil)
fig = figure(title="Covid-19 no Brasil",
             x_axis_type='datetime',
             width=1200,
             height=400,
             x_axis_label="Data",
             y_axis_label="Casos Acumulados")
fig.line("data",
         "casosAcumulado",
         source=source_brasil,
         color=Category10_6[0],
         legend_label="Acumulados")
fig.circle("data",
           "casosAcumulado",
           source=source_brasil,
           color=Category10_6[0],
           legend_label="Acumulados")
fig.line("data",
         "obitosAcumulado",
         source=source_brasil,
         color=Category10_6[1],
         legend_label="Óbitos")
fig.circle("data",
           "obitosAcumulado",
           source=source_brasil,
           color=Category10_6[1],
           legend_label="Óbitos")
fig.legend.location = "top_left"
fig.legend.click_policy = 'hide'
fig.add_tools(
    HoverTool(
        tooltips=[("Casos Acumulados",
                   "@casosAcumulado"), ("Óbitos Acumulados", "@obitosAcumulado"
                                       ), ("Semana Epidemiológica",
                                           "@semanaEpi")]))

show(fig)

## Plot de vários estados por data

In [14]:
# state_abrv_vec = np.array(['CE', 'DF', 'PE', 'RJ', 'SP', 'BA', 'AM', 'MS', 'RN'])
state_abrv_vec = np.array(['CE', 'DF', 'RJ', 'SP', 'AM'])


def get_plot(column_name, y_axis_label="Label", width=1200, height=400):

    fig = figure(title="Covid-19 nos estados",
                 x_axis_type='datetime',
                 width=width,
                 height=height,
                 x_axis_label="Data",
                 y_axis_label=y_axis_label)
    for i, sigla in enumerate(state_abrv_vec):
        state_data = data_estados[data_estados.estado==sigla]
        source = ColumnDataSource(state_data)
        fig.line("data",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category10_6[i])
        fig.circle("data",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category10_6[i])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(
            tooltips=[("Estado", "@estado"), ("Dia da Semana", "@diaDaSemana"),
                      ("Casos Acumulados",
                       "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
width = 650
height = 300
plot_casosAcumulados = get_plot("casosAcumulado",
                                y_axis_label="Casos Acumulados",
                                width=width,
                                height=height)
plot_casosNovos = get_plot("casosNovos",
                           y_axis_label="Casos Novos",
                           width=width,
                           height=height)
plot_obitosAcumulados = get_plot("obitosAcumulado",
                                 y_axis_label="Óbitos Acumulados",
                                 width=width,
                                 height=height)
plot_obitosNovos = get_plot("obitosNovos",
                            y_axis_label="Óbitos Novos",
                            width=width,
                            height=height)

show(
    grid([[plot_casosAcumulados, plot_casosNovos],
          [plot_obitosAcumulados, plot_obitosNovos]]))

## Plot de vários estados por dia de contaminação


In [22]:
# Estados com crescimento mais rápido por dia de contaminação
estados_crescimento_rapido = ["CE", "SP", "AM", "PE", "RJ"]


def get_plot_por_dia_de_contaminacao(column_name,
                                     y_axis_label="Label",
                                     min_casos=1, # Deve ser ou 1 ou 100
                                     width=1200,
                                     height=400,
                                    title="Covid-19 nos 5 estados com crescimento mais rápido por dia de contaminação"):
    fig = figure(
        title=title,
        width=width,
        height=height,
        x_axis_label="Dia de Contaminação",
        y_axis_label=y_axis_label)

    for i, sigla in enumerate(estados_crescimento_rapido):
        state_data = data_estados[data_estados.estado==sigla]
        source = ColumnDataSource(state_data)
        fig.line(f"diasDeContaminacao_{min_casos}",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category20_20[i % 20])
        fig.circle(f"diasDeContaminacao_{min_casos}",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category20_20[i % 20])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(
            tooltips=[(
                "Estado",
                "@estado"), ("Dias de contaminação", f"@diasDeContaminacao_{min_casos}"),
                      ("Dia da Semana",
                       "@diaDaSemana"), (
                           "Casos Acumulados",
                           "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


plot_casosAcumulados_por_dia_de_contaminacao_1 = get_plot_por_dia_de_contaminacao(
    "casosAcumulado", y_axis_label="Casos Acumulados", min_casos=1, width=width, height=500, title="Covid-19 por dia de contaminação do primeiro caso")
plot_casosAcumulados_por_dia_de_contaminacao_100 = get_plot_por_dia_de_contaminacao(
    "casosAcumulado", y_axis_label="Casos Acumulados", min_casos=100, width=width, height=500, title="Covid-19 por dia de contaminação do centésimo caso")
# plot_casosNovos_por_dia_de_contaminacao = get_plot_por_dia_de_contaminacao("casosNovos", y_axis_label="Casos Novos", width=width, height=height)
show(grid([[plot_casosAcumulados_por_dia_de_contaminacao_1, plot_casosAcumulados_por_dia_de_contaminacao_100]]))
# show(plot_casosAcumulados_por_dia_de_contaminacao_100)

<span style="color:red; font-weight: 700; font-size:20px;">A parte preocupande é que o Ceará tem um rítimo de crescimento tão rápido ou levemente maior que São Paulo. <br>
    A diferença é que o primeiro caso no Ceará foi 20 dias depois do primeiro caso em São Paulo.</span>

# Verificando se estamos saindo da crise

Um bom gráfico para saber se estamos saindo da crise é o de casos novos por casos existentes (ambos em escala log). Enquanto estivermos na epidemia o gráfico aproxima-se de uma reta. Quando começarmos a sair da epidemia o gráfico cai.

In [16]:
def get_plot_casos_novos_por_casos_acumulados(df, width=1200, height=400):
    fig = figure(title="Casos novos por casos acumulados",
                 width=width,
                 height=height,
                 x_axis_type="log",
                 y_axis_type="log",
                 x_axis_label="Casos Acumulados (log)",
                 y_axis_label="Casos Novos (log)")
    fig.line('casosAcumulado', "casosNovos", source=df)
    fig.circle('casosAcumulado', "casosNovos", source=df)
    fig.add_tools(
        HoverTool(
            tooltips=[("Dia da Semana",
                       "@diaDaSemana"), (
                           "Casos Acumulados",
                           "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


plot_casos_novos_por_acumulados = get_plot_casos_novos_por_casos_acumulados(
    data_brasil)
show(plot_casos_novos_por_acumulados)